# SUDOKU NAKED TWINS

Naked twins strategy consist

The naked twins technique is the following. Consider the following puzzle, and look at the two highlighted boxes, 'F3' and 'I3'.
<img src='imgs/naked-twins.png' width='300px'>
As we can see, both belong to the same column, and both permit the values of 2 and 3. Now, we don't know which one has a 2 and which one has a 3, but we know one thing for sure — the values 2 and 3 are locked in those two boxes, so no other box in their same unit (the third column) can contain the values 2 or 3.

Thus, we go over all the boxes in their same unit, and remove the values 2 and 3 from their possible values.
<img src='imgs/naked-twins-2.png' width='300px'>

As you can see, we've removed the values 2 and 3 from the boxes 'D3' and 'E3'. This is the naked twins technique. In this project, you'll write a function that implements this technique.

In [20]:
assignments = []


def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values


def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)

peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))


def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print



def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values


def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values


def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt


def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    
    
'''
In this case, we are still a long way from solving the puzzle--
61 squares remain uncertain. What next? We could try to code more sophisticated strategies. 
For example, the naked twins strategy looks for two squares in the same unit that both have 
the same two possible digits. Given {'A5': '26', 'A6':'26', ...}, we can conclude that 2 and 
6 must be in A5 and A6 (although we don't know which is where), and we can therefore eliminate 
2 and 6 from every other square in the A row unit. We could code that strategy in a few lines 
by adding an elif len(values[s]) == 2 test to eliminate.
'''
    
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    twins = [] #keep record of visited twins
    for box in boxes:
        
        for peer in peers[box]:
            if values[peer] == values[box] and len(values[peer])==2:
                if ([box, peer] not in twins) and ([peer, box] not in twins):
                    twins.append([box, peer])
                    #find shared peers between twin (box and peer)
                    shared_peers = peers(box).intersection(peers(peer))
                    
                    #iterate and =
                    # Eliminate the naked twins as possibilities for their peers
                    values_box = [val for val in values[box]]
                    for shared_p in shared_peers:
                        if len(values[shared_p]) > 2: #skip peers with less than 2 elements
                            del_status = True #is True is all digits of twins are in shared_peer
                            for digit in values_box:
                                if digit in values[shared_p]:
                                    del_status = del_status * True
                                else:
                                    del_status = del_status * False
                            if del_status:
                                for digit in values_box:
                                    values[shared_p].replace(digit, '')
    
    return values

    
if __name__ == '__main__':
    
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    
    values = grid_values(diag_sudoku_grid)
    ng = reduce_puzzle(values)
    display(ng)
    naked_twins(ng)
    print(ng)

    #display(ng)

    #display(solve(diag_sudoku_grid))

    #try:
    #    from visualize import visualize_assignments
    #    visualize_assignments(assignments)

    #except SystemExit:
    #    pass
    #except:
    #    print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


   2     356789  34789  |1345789  134578  134579 | 13569  1345689  145689 
 45789   35789   34789  |1345789  134578    6    |   2     134589  14589  
 45689   35689     1    | 34589   23458   23459  |  3569     7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    | 13457   123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357   12357  |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      2379  | 13579   13567   13579  |   8     12569   12569  
 16789   136789    5    |   2     134678  13479  |  1679    1469    1469  
 16789   126789   2789  | 145789  145678  14579  | 15679   124569    3    
{'I1': '16789', 'G7': '8', 'E6': '1245', 'E1': '3', 'E4': '145', 'F5': '12357', 'H3': '5', 'B1': '45789', 'E9': '7', 'H4': '2', 'D9': '1259', 'A5': '134578', 'D6': '8', 'I8': 

In [10]:
x = {'A8', 'A1', 'C3', 'H3', 'B1', 'A5', 'A7', 'G3', 'E3', 'C2', 'A4', 'A6', 'A2', 'F3', 'I3', 'B3', 'D3', 'B2', 'C1', 'A9'}
y={'A1', 'C3', 'B1', 'H3', 'B4', 'B7', 'B6', 'G3', 'E3', 'C2', 'I3', 'A2', 'F3', 'D3', 'B2', 'A3', 'C1', 'B5', 'B8', 'B9'}
t = x.intersection(y)
print(t)

{'G3', 'E3', 'C2', 'I3', 'A2', 'F3', 'A1', 'D3', 'H3', 'B1', 'C3', 'B2', 'C1'}


In [18]:
a = '34567890'
b='30'
if each(b) in a:
    print(b)

NameError: name 'each' is not defined

ValueError: empty separator